In [1]:
# 1) Design model (input,output,forward pass)
# 2) Construct loss and optimizer 
# 3) Trainning Loop
# -foward pass: compute prediction
# -backward pass:gradients
# -update weights

In [8]:
import torch
import torch.nn as nn # neural network module
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
%matplotlib inline

In [17]:
# 0) Data Preparation

bc = datasets.load_breast_cancer()
X, y = bc.data, bc.target

n_samples, n_features = X.shape
print(n_samples,n_features)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=5)

# Scaled
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

y_train = y_train.view(y_train.shape[0], 1)  # .view = .reshape
y_test = y_test.view(y_test.shape[0], 1)

569 30


In [20]:
# 1) Design Model
# f = wx+b then sigmoid function
class LogisticRegression(nn.Module):
    def __init__(self,n_input_features):
        super(LogisticRegression,self).__init__()
        self.linear = nn.Linear(n_input_features,1) # 1 is output
    
    def forward(self,x):
        y_predicted = torch.sigmoid(self.linear(x))
        return y_predicted
    
model = LogisticRegression(n_features)


In [21]:
#Loss and optimizier
learning_rate = 0.01

criterion = nn.BCELoss() # Binary Crossentropy
optimizer = torch.optim.SGD(model.parameters(),lr=learning_rate)

In [25]:
#Trainning Loop
EPOCHS = 100

for epoch in range(EPOCHS):
    #foward pass and loss
    y_predicted = model(X_train)
    loss = criterion(y_predicted,y_train)
    # Backward pass
    loss.backward()
    #update weights
    optimizer.step()
    # empty gradients zero it out
    optimizer.zero_grad()
    if (epoch+1)%10 ==0:
        print(f"epoch: {epoch+1}, loss ={loss.item():.4f}")
        
        
# Model Evaulating
with torch.no_grad():
    y_predicted = model(X_test)
    y_predicted_cls = y_predicted.round()
    accuracy = y_predicted_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f"accuracy = {accuracy:.4f}")

epoch: 10, loss =0.1557
epoch: 20, loss =0.1537
epoch: 30, loss =0.1518
epoch: 40, loss =0.1500
epoch: 50, loss =0.1483
epoch: 60, loss =0.1466
epoch: 70, loss =0.1450
epoch: 80, loss =0.1435
epoch: 90, loss =0.1420
epoch: 100, loss =0.1406
accuracy = 0.9737
